### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [6]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3220601979054173  /  0.03235479719328581
estimators:  10  prof:  5  | R2 mean / std:  0.39673976870313626  /  0.0372469875653716
estimators:  10  prof:  10  | R2 mean / std:  0.3746480574947064  /  0.07979002198064278
estimators:  100  prof:  1  | R2 mean / std:  0.32739700789873477  /  0.049535801337424856
estimators:  100  prof:  5  | R2 mean / std:  0.42303621385857  /  0.04572536518465193
estimators:  100  prof:  10  | R2 mean / std:  0.4210300869333083  /  0.034687165484204495
estimators:  100  prof:  1  | R2 mean / std:  0.3395713797338344  /  0.03473675158358899
estimators:  100  prof:  5  | R2 mean / std:  0.42620124945898147  /  0.027911932408408315
estimators:  100  prof:  10  | R2 mean / std:  0.4244885559226894  /  0.055098078670731965


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

In [8]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [9]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [10]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [11]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [12]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [13]:
grid.best_score_

0.4245301970158746

In [14]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [15]:
grid.cv_results_

{'mean_fit_time': array([0.03825649, 1.9897027 , 0.02280442, 2.44148286]),
 'std_fit_time': array([0.00237028, 0.25531364, 0.00075581, 0.04154134]),
 'mean_score_time': array([0.00388145, 0.11466996, 0.00225131, 0.1170191 ]),
 'std_score_time': array([0.00042433, 0.00660699, 0.00038505, 0.00417892]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'split0_test_score': array([0.35959734, 0.35356916, 0.3122179 , 0.37014976]),
 'split1_test_score': array([0.41700395, 0.44741653, 0.37012753, 0.4534257 ]),
 'split2_test_score': array([0.41816742, 0.4

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

In [16]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [17]:
df.head()

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [18]:
df.corr()['preco_mediano_das_casas']

longitude                 -0.045967
latitude                  -0.144160
idade_mediana_das_casas    0.105623
total_comodos              0.134153
total_quartos              0.049686
populacao                 -0.024650
familias                   0.065843
salario_mediano            0.688075
preco_mediano_das_casas    1.000000
Name: preco_mediano_das_casas, dtype: float64

In [19]:
df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [20]:
df.dropna(inplace=True)

In [21]:
df.isnull().sum()

longitude                  0
latitude                   0
idade_mediana_das_casas    0
total_comodos              0
total_quartos              0
populacao                  0
familias                   0
salario_mediano            0
preco_mediano_das_casas    0
proximidade_ao_mar         0
dtype: int64

In [22]:
variaveis_para_normalizar = ['idade_mediana_das_casas',
                             'total_comodos',
                             'total_quartos',
                             'populacao',
                             'familias',
                             'salario_mediano']



#### Normalizando os dados

In [23]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [24]:
ohe=OneHotEncoder()
Sc=StandardScaler()
df_ohe=ohe.fit_transform(df[["proximidade_ao_mar"]]).toarray()
df_sc=Sc.fit_transform(df[variaveis_para_normalizar])
x,y=pd.DataFrame(np.c_[df_ohe,df_sc]), df.preco_mediano_das_casas.values

In [25]:
x.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,0.0,1.0,0.0,0.982163,-0.803813,-0.970325,-0.973320,-0.976833,2.345163
1,0.0,0.0,0.0,1.0,0.0,-0.606210,2.042130,1.348276,0.861339,1.670373,2.332632
2,0.0,0.0,0.0,1.0,0.0,1.855769,-0.535189,-0.825561,-0.819769,-0.843427,1.782939
3,0.0,0.0,0.0,1.0,0.0,1.855769,-0.623510,-0.718768,-0.765056,-0.733562,0.932970
4,0.0,0.0,0.0,1.0,0.0,1.855769,-0.461970,-0.611974,-0.758879,-0.628930,-0.013143


In [26]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

#### Instanciando os algoritmos

##### Gradient Boosting

In [27]:
gb=GradientBoostingRegressor()
param_gb={"n_estimators":[10,100,1000],"learning_rate":[0.01,0.1,1],"alpha":[0.1,0.3,0.5]}

In [28]:
grid_search=GridSearchCV(estimator=gb,param_grid=param_gb)

In [35]:
grid_search.fit(x_train,y_train)

GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'alpha': [0.1, 0.3, 0.5],
                         'learning_rate': [0.01, 0.1, 1],
                         'n_estimators': [10, 100, 1000]})

In [37]:
grid_search.best_params_

{'alpha': 0.5, 'learning_rate': 0.1, 'n_estimators': 1000}

In [39]:
grid_search.best_score_

0.7119818612807757

##### KNN

In [41]:
knn=KNeighborsRegressor()
param_knn={"n_neighbors":[2,5,10,20]}

In [43]:
grid_search_knn=GridSearchCV(estimator=knn,param_grid=param_knn)

In [69]:
grid_search_knn.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [2, 5, 10, 20]})

In [70]:
grid_search_knn.best_params_

{'n_neighbors': 20}

In [71]:
grid_search_knn.best_score_

0.6859970795691427

##### Random Forest

In [72]:
rf=RandomForestRegressor()
param_rf={"n_estimators":[10,100,500,1000],"max_depth":[2,5,7,15]}
grid_search_rf=GridSearchCV(estimator=rf,param_grid=param_rf)

In [73]:
grid_search_rf.fit(x_train,y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 5, 7, 15],
                         'n_estimators': [10, 100, 500, 1000]})

In [74]:
grid_search_rf.best_params_

{'max_depth': 15, 'n_estimators': 1000}

In [75]:
grid_search_rf.best_score_

0.7117730008052614

### Criando a classe Grid Search

__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

In [76]:
class gridSearchAll():
    
    def __init__(self,scoring,num_folds):
      self.grid_models = []
      self.best=[]
      self.scoring=scoring
      self.num_folds=num_folds
    
    def insert_model(self, estimator_base, param_grid):
      self.grid_models.append([estimator_base, param_grid])
        
    def fit_all(self,x,y):
      for i in self.grid_models:
        gsa=GridSearchCV(estimator=i[0],param_grid=i[1],scoring=self.scoring,cv=self.num_folds)
        gsa.fit(x,y)
        self.best.append([gsa.best_score_,gsa.best_estimator_])
      self.best_all_grid_models=self.best
    
    def best_all_grid_models(self):
      df=pd.DataFrame(self.best_all_grid_models)
      return  df

In [77]:
g_s=gridSearchAll("r2",3)

In [78]:
g_s.insert_model(gb,param_gb)
g_s.insert_model(knn,param_knn)
g_s.insert_model(rf,param_rf)

In [84]:
g_s.fit_all(x_train,y_train)

In [85]:
g_s.best_all_grid_models

[[0.7089546263587742, GradientBoostingRegressor(alpha=0.5, n_estimators=1000)],
 [0.6823760607616697, KNeighborsRegressor(n_neighbors=10)],
 [0.7093694825274529, RandomForestRegressor(max_depth=15, n_estimators=500)],
 [0.7089156806800263, GradientBoostingRegressor(alpha=0.3, n_estimators=1000)],
 [0.6823760607616697, KNeighborsRegressor(n_neighbors=10)],
 [0.70939441230739, RandomForestRegressor(max_depth=15, n_estimators=500)]]

### Com o retorno da  classe,  pude verificar que o melhor modelo nos dados de treino foi a random foresy,apresentando um erro menor. Irei aplicar nos dados de testes para comprovar.  

In [86]:
from sklearn.metrics import r2_score

In [87]:
gb.fit(x_train,y_train)

GradientBoostingRegressor()

In [88]:
r2_score(y_test,gb.predict(x_test))

0.7083921340398349

In [89]:
knn.fit(x_train,y_train)

KNeighborsRegressor()

In [90]:
r2_score(y_test,knn.predict(x_test))

0.6916298915562478

In [91]:
rf.fit(x_train,y_train)

RandomForestRegressor()

In [92]:
r2_score(y_test,rf.predict(x_test))

0.7180803789906534

## como podemos ver, o Random Forest se saiu bem apresentando erros muitos parecidos com os dados de treino, descartando assim a ocorrência de overfitting